# Verificacion de Estimaciones para la Campaña 2023.

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_32079/1758670802.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Muestreo_CT_PatAzul2023.csv')


In [3]:
df.columns

Index(['Unnamed: 0', 'Ruta', 'Sitio', 'Año', 'Camara', 'Extra', 'Archivo',
       'Fecha', 'Hora', 'Animal_proba', 'Animal', 'Guanaco_proba', 'Guanaco',
       'Especie', 'Cantidad_proba', 'Cantidad', 'Validar', 'Validado'],
      dtype='object')

## Estimacion de Proporciones:

In [6]:
# Proporcion de Imagenes con Animales
pAnimal = sum(df['Animal']==1)/len(df)
print('Proporcion de Imagenes con Animales: ', round(pAnimal * 100,2), '%')

Proporcion de Imagenes con Animales:  29.2 %


In [7]:
# Proporcion de Imagenes sin Animales
pNoAnimal = sum(df['Animal']==0)/len(df)
print('Proporcion de Imagenes sin Animales: ', round(pNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales:  70.8 %


## Tiempo Total del Proceso:

### Tiempo de Ejecucion Automatica:

La eficiencia en los tiempos se vio perjudica por el uso de AWS para almacenar las imagenes. Al tener mayor latencia el proceso de consultar las imagenes demora más. Ademas, para consultar las imagenes se debieron tensorificar y extraer los metadatos antes de aplicar los modelos.

Se utilizaron multiprocesos concurrentes para hacer el proceso mas rapido. Y tecnicas de autoguardado para que en caso de que se rompa el proceso no se tenga que iniciar desde cero.

- Tensorificacion: 1.3 imagenes por segundo, por lo que el proceso completo llevo 25.4 horas.
- Extraccion de Metadata: 2.39 imagenes por segundo, por lo que el proceso completo llevo 13.8 horas.
- Modelos: XXXX


In [8]:
tTensorifizacion = 1.3
tMetadata = 2.39
tModelos = 6.25 # CAMBIAR
tAutom =(1/tTensorifizacion)+(1/tMetadata)+(1/tModelos)
TAutom = (len(df) * (tAutom))/(60*60)
print('Tiempo de Automatizacion: ', round(TAutom,2), 'horas')
print('Tiempo de Automatizacion: ', math.ceil(TAutom/8), 'jornadas')

Tiempo de Automatizacion:  44.46 horas
Tiempo de Automatizacion:  6 jornadas


Por lo que el tiempo total del proceso automatico es de **XX horas**  que son equivalentes a **XX jornadas**.

### Tiempo de Validacion:

El porcantaje de imagenes a validar sobre el total de la base es de:

In [9]:
pValNoAnimal = sum((df['Validar']==1) & (df['Animal']==0))/sum(df['Animal']==0)
print('Proporcion de Imagenes sin Animales a Validar: ', round(pValNoAnimal * 100,2), '%')

Proporcion de Imagenes sin Animales a Validar:  13.93 %


In [10]:
pValAnimal = sum((df['Validar']==1) & (df['Animal']==1))/sum(df['Animal']==1)
print('Proporcion de Imagenes con Animales a Validar: ', round(pValAnimal * 100,2), '%')

Proporcion de Imagenes con Animales a Validar:  79.58 %


In [11]:
# Porcentaje de Imagenes a Validar:
pVal = sum(df['Validar']==1)/len(df)
print('Porcentaje de Imagenes a Validar: ', round(pVal * 100,2), '%')

Porcentaje de Imagenes a Validar:  33.1 %


In [12]:
# Tiempo de Validacion:
tVal = pVal * 1.36
TVal = (len(df) * tVal)/(60*60)
print('Tiempo de Validacion: ', round(TVal,2), 'horas')
print('Tiempo de Validacion: ', math.ceil(TVal/8), 'jornadas')

Tiempo de Validacion:  14.85 horas
Tiempo de Validacion:  2 jornadas


Por lo que el tiempo total del proceso de validacion es de **XX horas**  que son equivalentes a **XX jornadas**.

### Tiempo Total

El tiempo total del proceso sera de:

In [54]:
TTotal = TAutom + TVal
print('Tiempo del Proceso: ', round(TTotal,2), 'horas')
print('Tiempo del Proceso: ', round(TTotal/8,2), 'jornadas')

Tiempo del Proceso:  59.31 horas
Tiempo del Proceso:  7.41 jornadas


El proceso completo requiere **XX jornadas laborales**  (XX horas). Cabe destacar que de estas solo **XX horas requieren atencion humana**

### Cantidad de Imagenes por Hora:

Para calcular la cantidad de imágenes que se procesaran por hora se tiene en cuenta tanto el tiempo que tarda en procesarlas los modelos predictivos cómo el tiempo que se tarda en validar aquellas que necesiten un refuerzo en su clasificación. 

In [14]:
print('Tiempo de Automatizacion: ', round(tAutom,2), 'segundos')

cHora = 60*60/(tAutom + tVal)
print('Cantidad de Imagenes por Hora: ', math.ceil(cHora))

Tiempo de Automatizacion:  1.35 segundos
Cantidad de Imagenes por Hora:  2003


### Precision de Etiquetado:

In [15]:
eNoAnimal = pNoAnimal * (95.94/100) * (1.45/100)
print('Error de Imagenes sin Animales: ', round(eNoAnimal * 100,2), '%')

Error de Imagenes sin Animales:  0.98 %


In [16]:
eAnimal = pAnimal * (49.19/100) * ((1.41/100) + (41.79/100) * (5.22/100))
print('Error de Imagenes con Animales: ', round(eAnimal * 100,2), '%')

Error de Imagenes con Animales:  0.52 %


In [17]:
eAutomatico = eNoAnimal + eAnimal
print('Error Automatico: ', round(eAutomatico * 100,2), '%')

Error Automatico:  1.5 %


In [40]:
cErroresAutom = len(df) * eAutomatico
print('Cantidad de Errores: ', math.ceil(cErroresAutom))

Cantidad de Errores:  1783


In [42]:
eValidacion = pAnimal * ((16.7/100) + (3.4/100) + (10/100))/2 * pValAnimal + pNoAnimal * ((0.6/10) /2) * pValNoAnimal
print('Error de Validacion: ', round(eValidacion * 100,2), '%')

Error de Validacion:  3.79 %


In [43]:
cErroresVal = len(df) * pVal * eValidacion 
print('Cantidad de Errores: ', math.ceil(cErroresVal))

Cantidad de Errores:  1492


In [44]:
eErrores = eAutomatico + eValidacion
print('Error Total: ', round(eErrores * 100,2), '%')

Error Total:  5.29 %


In [45]:
cErrores = cErroresAutom + cErroresVal
print('Cantidad de Errores: ', math.ceil(cErrores))

Cantidad de Errores:  3274


### Costos del Proceso:

- Costo Falso Negativo: $300
- Costo Falso Positivo: $1000
- Costo Error en Especies: $1300
- Costo Error en Cantidad: $650

In [19]:
coErrorNoAnimal = (0.95/100) * 300
print('Costo de Error de Imagenes sin Animales: $', round(coErrorNoAnimal,2))

Costo de Error de Imagenes sin Animales: $ 2.85


In [20]:
coErrorAnimal = pAnimal * (49.19/100) * ((1.41/100) * 1300 + (41.79/100) * (5.22/100) * 650)
print('Costo de Error de Imagenes con Animales: $', round(coErrorAnimal,2))

Costo de Error de Imagenes con Animales: $ 4.67


In [21]:
coErrorAutomatico = coErrorNoAnimal + coErrorAnimal
print('Costo de Error Automatico: $', round(coErrorAutomatico,2))

Costo de Error Automatico: $ 7.52


In [22]:
coErrorVal = pAnimal * pValAnimal * ((1.41/100) * 300 + (41.79/100) * (5.22/100) * 650)/2 + pNoAnimal * pValNoAnimal * ((0.6/100)/2) * 1000
print('Costo de Error de Validacion: $', round(coErrorVal,2))

Costo de Error de Validacion: $ 2.44


In [23]:
coOperacionalVal = pVal * tVal * 10730/(8*60*60)
print('Costo Operacional de Validacion: $', round(coOperacionalVal,2))

Costo Operacional de Validacion: $ 0.06


In [24]:
coServicio = 1100000
print('Costo de Servicio: $', round(coServicio,2))

Costo de Servicio: $ 1100000


In [25]:
coTotal = len(df) * (coErrorAutomatico + coErrorVal + coOperacionalVal) + coServicio
print('Costo Total: $', round(coTotal,2))

Costo Total: $ 2288940.93


## KPIs

### Tiempo Total del Proceso

In [26]:
print('Tiempo Total: ')
print('- Clasificacion Manual: 152.75 horas')
print('- Solucion Semi-Automatica:', round(TTotal,2), 'horas')
print('Variacion Porcentual: ', round((TTotal - 152.75)/152.75 * 100,2), '%')

Tiempo Total: 
- Clasificacion Manual: 152.75 horas
- Solucion Semi-Automatica: 59.31 horas
Variacion Porcentual:  -61.17 %


In [27]:
print('Jornadas Laborales:')
print('- Clasificacion Manual: 19.1 jornadas')
print('- Solucion Semi-Automatica:')
print('   - Jornadas Ocupadas: ', round(TVal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TVal/8 - 19.1)/19.1 * 100,2), '%')
print('    - Jornadas Totales: ', round(TTotal/8,1), 'jornadas')
print('    Variacion Porcentual: ', round((TTotal/8 - 19.1)/19.1 * 100,2), '%')

Jornadas Laborales:
- Clasificacion Manual: 19.1 jornadas
- Solucion Semi-Automatica:
   - Jornadas Ocupadas:  1.9 jornadas
    Variacion Porcentual:  -90.28 %
    - Jornadas Totales:  7.4 jornadas
    Variacion Porcentual:  -61.18 %


### Cantidad de Imagenes Analizadas por Hora:

In [28]:
print('Cantidad de Imagenes por Hora: ')
print('- Clasificacion Manual: 777 imagenes/hr')
print('- Solucion Semi-Automatica: ', math.ceil(cHora), 'imagenes/hr')
print('Variacion Porcentual: ', round((cHora - 777)/777 * 100,2), '%')

Cantidad de Imagenes por Hora: 
- Clasificacion Manual: 777 imagenes/hr
- Solucion Semi-Automatica:  2003 imagenes/hr
Variacion Porcentual:  157.71 %


### Precision de Etiquetado:

In [48]:
print('Tasa de Error: ')
print('- Clasificacion Manual: 10%')
print('- Solucion Semi-Automatica: ', round(eErrores * 100,2), '%')
print('Variacion Porcentual: ', round((eErrores - 10)/10 * 100,2), '%')

Tasa de Error: 
- Clasificacion Manual: 10%
- Solucion Semi-Automatica:  5.29 %
Variacion Porcentual:  -99.47 %


In [50]:
print('Cantidad de Errores: ')
print('- Clasificacion Manual: 11877 errores')
print('- Solucion Semi-Automatica: ', math.ceil(cErrores), 'errores')
print('Variacion Porcentual: ', round((cErrores - 11877)/11877 * 100,2), '%')

Cantidad de Errores: 
- Clasificacion Manual: 11877 errores
- Solucion Semi-Automatica:  3274 errores
Variacion Porcentual:  -72.44 %


### Costo del Proceso:

In [29]:
print('Costo del Proceso:')
print('- Clasificacion Manual: $ 6722325.4')
print('- Solucion Semi-Automatica: $', round(coTotal,2))
print('Variacion Porcentual: ', round((coTotal - 6722325.4)/6722325.4 * 100,2), '%')

Costo del Proceso:
- Clasificacion Manual: $ 6722325.4
- Solucion Semi-Automatica: $ 2288940.93
Variacion Porcentual:  -65.95 %


### Ratio de Imagenes Utiles:

In [30]:
print('Ratio de Imagenes Utiles: ')
print('- Clasificacion Manual: 0.79%')
print('- Solucion Semi-Automatica: 100%')
print('Variacion Porcentual: ', round((100 - 0.79)/0.79 * 100,2), '%')

Ratio de Imagenes Utiles: 
- Clasificacion Manual: 0.79%
- Solucion Semi-Automatica: 100%
Variacion Porcentual:  12558.23 %


# Resultados:

In [57]:
# Creación del DataFrame
data = {
    "KPI": [
        "Horas Totales",
        "Jornadas Ocupadas",
        "Jornadas Totales",
        "Imagenes por Hora", 
        "Tasa de Error", 
        "Cantidad de Errores", 
        "Costo del Proceso", 
        "Ratio de Imagenes Utiles"
    ],
    "Clasificacion Manual": [
        152.75, # Tiempo Total
        19.1, # Jornadas Ocupadas
        19.1, # Jornadas Totales
        777, # Imagenes por Hora
        10, # Tasa de Error
        11877, # Cantidad de Errores
        6722325.4, # Costo del Proceso
        0.79 # Ratio de Imagenes Utiles
    ],
    "Estimacion Semi-Automatica": [
        60.71, # Tiempo Total
        1.6, # Jornadas Ocupadas
        round(60.7/8,2), # Jornadas Totales
        1957, # Imagenes por Hora
        5.4, # Tasa de Error
        3113, # Cantidad de Errores
        4944552.53, # Costo del Proceso
        100 # Ratio de Imagenes Utiles
    ],
    "Solucion Semi-Automatica": [
        round(TTotal,2), 
        round(TVal/8,1), 
        round(TTotal/8,1), 
        math.ceil(cHora), 
        round(eErrores * 100,2), 
        math.ceil(cErrores), 
        round(coTotal,2), 
        100 # Ratio de Imagenes Utiles
    ]
}

df = pd.DataFrame(data)
df["Variacion Porcentual (%)"] = round((df["Solucion Semi-Automatica"] - df["Clasificacion Manual"]) / df["Clasificacion Manual"] * 100, 2)
df

,KPI,Clasificacion Manual,Estimacion Semi-Automatica,Solucion Semi-Automatica,Variacion Porcentual (%)
0,Horas Totales,152.75,60.71,59.31,-61.17
1,Jornadas Ocupadas,19.10,1.60,1.90,-90.05
2,Jornadas Totales,19.10,7.59,7.40,-61.26
3,Imagenes por Hora,777.00,1957.00,2003.00,157.79
4,Tasa de Error,10.00,5.40,5.29,-47.10
5,Cantidad de Errores,11877.00,3113.00,3274.00,-72.43
6,Costo del Proceso,6722325.40,4944552.53,2288940.93,-65.95
7,Ratio de Imagenes Utiles,0.79,100.00,100.00,12558.23
